In [2]:
import sys
import Bio
from Bio import SeqIO, SeqFeature
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import os
import pandas as pd
import subprocess
import re
from Promoter.DeepTFactor.test_deeptfactor.deeptftest import deeptfactor

In [3]:
genome_path = "/home/justin/Master/Data_collection/Genome/test.fasta"
df = pd.DataFrame(columns = ['Gene loci', 'Bprom', 'DeepTFactor'])

In [4]:
def bprom(gene_outpath, bprom_out):
    
    subprocess.run("export TSS_DATA=/home/justin/Master/Attribute_prediction/Promoter/Bprom/bprom_run/data ; /home/justin/Master/Attribute_prediction/Promoter/Bprom/bprom_run/bprom %s %s" % (gene_outpath, bprom_out), shell=True)

In [10]:
#var1 = "/home/justin/Master/Attribute_prediction/Promoter/Bprom/doc/seq.fa"
#bprom_out = "/home/justin/Master/Attribute_prediction/Promoter/Bprom/bprom_run/test.txt"

#subprocess.run("export TSS_DATA=/home/justin/Master/Attribute_prediction/Promoter/Bprom/bprom_run/data ; /home/justin/Master/Attribute_prediction/Promoter/Bprom/bprom_run/bprom %s %s" % (gene_outpath, bprom_out), shell=True)

CompletedProcess(args='export TSS_DATA=/home/justin/Master/Attribute_prediction/Promoter/Bprom/bprom_run/data ; /home/justin/Master/Attribute_prediction/Promoter/Bprom/bprom_run/bprom /home/justin/Master/Attribute_prediction/Promoter/Bprom/doc/seq.fa /home/justin/Master/Attribute_prediction/Promoter/Bprom/bprom_run/test.txt', returncode=0)

In [34]:
# def deeptfactor(protein_outpath, deeptf_out):
    
#     subprocess.run("cd /home/justin/Master/Attribute_prediction/Promoter/DeepTFactor/deeptfactor ; conda run -n deeptfactor python /home/justin/Master/Attribute_prediction/Promoter/DeepTFactor/deeptfactor/tf_running.py -i %s -o %s -g cpu" % (protein_outpath, deeptf_out), shell=True)

In [5]:
#subprocess.run("export TSS_DATA=/home/justin/Master/Attribute_prediction/Promoter/Bprom/bprom_run/data ; /home/justin/Master/Attribute_prediction/Promoter/Bprom/bprom_run/bprom /home/justin/Master/Data_collection/Genome/test.fasta /home/justin/Master/Attribute_prediction/Promoter/Bprom/bprom_run/test.txt", shell=True)
#subprocess.run("/home/justin/Master/Attribute_prediction/Promoter/Bprom/bprom_run/bprom /home/justin/Master/Data_collection/Genome/test.fasta /home/justin/Master/Attribute_prediction/Promoter/Bprom/bprom_run/test.txt", shell=True)

#subprocess.run("cd /home/justin/Master/Attribute_prediction/Promoter/DeepTFactor/deeptfactor ; conda run -n deeptfactor python /home/justin/Master/Attribute_prediction/Promoter/DeepTFactor/deeptfactor/tf_running.py -i /home/justin/Master/Data_collection/Genome/NC_006350.1_full_CDS_translation.fasta -o /home/justin/Master/Attribute_prediction/Promoter/DeepTFactor/deeptfactor/result -g cpu", shell=True)

CompletedProcess(args='cd /home/justin/Master/Attribute_prediction/Promoter/DeepTFactor/deeptfactor ; conda run -n deeptfactor python /home/justin/Master/Attribute_prediction/Promoter/DeepTFactor/deeptfactor/tf_running.py -i /home/justin/Master/Data_collection/Genome/NC_006350.1_full_CDS_translation.fasta -o /home/justin/Master/Attribute_prediction/Promoter/DeepTFactor/deeptfactor/result -g cpu', returncode=0)

In [ ]:
##previous test code
#for gene in SeqIO.parse(genome_path,"fasta"):
#    print(gene.id)
#    print(gene.seq)
#     SeqIO.write(gene, "generecord.fasta", "fasta")
#     bprom()
#     deeptfactor()
#     bprom_result = open("/home/justin/Master/Attribute_prediction/Promoter/Bprom/bprom_run/test.txt").read()
#     df = df.append({'Gene loci' : gene.id, 'Sequence' : str(gene.seq), 'Bprom' : bprom_result}, ignore_index = True)

# print(df)

In [5]:
# declaring gb filepath, parsed file output path, tool output path
gb_path = "/home/justin/Master/Data_collection/Genome/NC_006350.1_full.gb"
gene_outpath = "/home/justin/Master/Attribute_prediction/generecord.fasta"
protein_outpath = "/home/justin/Master/Attribute_prediction/proteinrecord.fasta"
bprom_out = "/home/justin/Master/Attribute_prediction/Promoter/Bprom/bprom_run/test.txt"
# deeptf_out = "/home/justin/Master/Attribute_prediction/Promoter/DeepTFactor/deeptfactor/result/"
# deeptf_out_results = "/home/justin/Master/Attribute_prediction/Promoter/DeepTFactor/deeptfactor/result/prediction_result.txt"

In [12]:
#parsing
seq_record = next(SeqIO.parse(open(gb_path), "gb")) #not sure if next() is neccesary here yet
max_length = len(seq_record) #for reverse strands

#retrieve nucleotide and protein sequences from each CDS
for feature in seq_record.features:
    if feature.type == "CDS":
        #nucleotide sequence
        #obtain gene coordinates
        mystart = feature.location.start.position
        myend = feature.location.end.position 

        #obtain gene locus
        locus_key = 'old_locus_tag'

        if locus_key in feature.qualifiers:
            locus = ''.join(feature.qualifiers[locus_key])
        else:
            locus = ''.join(feature.qualifiers['locus_tag'])

        #obtain gene strand info, reverse complement the antisense strand
        if feature.strand == -1:
            this_start = mystart
            this_end = myend
            if this_end + 150 <= max_length:
                this_end = myend + 150 #append upstream gene
            # else:
            #     this_end = myend  
            gene_seq_upstream = seq_record.seq[this_start:this_end]
            dna_seq = gene_seq_upstream.reverse_complement()
        else:
            this_start = mystart
            this_end = myend
            if this_start - 150 >= 0:
                this_start = mystart - 150 #append upstream gene
            # else:
            #     this_start = mystart
            gene_seq_upstream = seq_record.seq[this_start:this_end]
            dna_seq = gene_seq_upstream 
        
        #gene sequence file creation and BPROM
        gene_seq_upstream = seq_record.seq[this_start:myend]
        geneseq = SeqRecord(dna_seq, id="%s" % (locus))
        SeqIO.write(geneseq, open(gene_outpath, "w"), "fasta")   

        bprom(gene_outpath = gene_outpath,bprom_out = bprom_out)
     
        ######protein sequence######################################################
        translation_key = 'translation'

        if translation_key in feature.qualifiers:
            translated_seq = ''.join(feature.qualifiers['translation'])
        # else:               #genes without translation info are pseudogenes, we will not translate it
            # trans_gene_seq = seq_record.seq[mystart:myend]
            # translated_seq = trans_gene_seq.translate(table=11, to_stop=True)
                
            if len(translated_seq) > 1000:
                ##slicing seq >1000 nt to 1000 nt in a list. If TF is found break from loop
                translated_seq_list = [translated_seq[i:i+1000] for i in range(len(translated_seq)-999)]
                for seq in translated_seq_list:
                    deeptf_result = deeptfactor(seq, locus)

                    if deeptf_result == 'True':
                        break
            else:
                deeptf_result = deeptfactor(translated_seq, locus)
        else:
            deeptf_result = 'pseudogene'

        # [Deprecated] protein sequence file creation and Deeptfactor
        # proteinseq = SeqRecord(Seq(translated_seq,), id="%s" % (locus), description = '')
        # SeqIO.write(proteinseq, open(protein_outpath, "w"), "fasta") 

        # deeptf_result = deeptfactor(translated_seq, locus)

        #append results in dataframe
        bprom_result_list = list(open(bprom_out).read().replace('  ','').split("\n"))
        bprom_result = bprom_result_list[3]
        #########need a method to parse TF binding site is available

        # deeptf_result_list = list(open(deeptf_out_results).read().replace('\n','').split("\t"))
        # deeptf_result = deeptf_result_list[3]
        
        df = df.append({'Gene loci' : locus, 'Bprom' : bprom_result, 'DeepTFactor' : deeptf_result}, ignore_index = True)

print(df)

        Gene loci                              Bprom DeepTFactor
0        BPSL0001   Number of predicted promoters -1       False
1        BPSL0002   Number of predicted promoters -2       False
2        BPSL0003   Number of predicted promoters -1       False
3        BPSL0004   Number of predicted promoters -1       False
4        BPSL0005   Number of predicted promoters -0       False
...           ...                                ...         ...
5565  BPS_RS18510   Number of predicted promoters -1       False
5566     BPSL3429   Number of predicted promoters -2       False
5567     BPSL3430   Number of predicted promoters -0       False
5568     BPSL3431   Number of predicted promoters -1        True
5569  BPS_RS18530   Number of predicted promoters -0       False

[5570 rows x 3 columns]


In [13]:
df.to_csv('df.csv', index = False)

In [38]:
print(proteinseq)

ID: BPSL0043
Name: <unknown name>
Number of features: 0
Seq('MQLHFESDLDYQLEAIEAVCDLFRGQEACRAEFSVTAQAARRRAGPQISLGMAE...DAA')


In [30]:
deeptfactor(protein_outpath = protein_outpath, deeptf_out = deeptf_out)

In [15]:
subprocess.run("cd /home/justin/Master/Attribute_prediction/Promoter/DeepTFactor/deeptfactor ; conda run -n deeptfactor python /home/justin/Master/Attribute_prediction/Promoter/DeepTFactor/deeptfactor/tf_running.py -i /home/justin/Master/Attribute_prediction/proteinrecord.fasta -o /home/justin/Master/Attribute_prediction/Promoter/DeepTFactor/deeptfactor/result -g cpu", shell=True)

CompletedProcess(args='cd /home/justin/Master/Attribute_prediction/Promoter/DeepTFactor/deeptfactor ; conda run -n deeptfactor python /home/justin/Master/Attribute_prediction/Promoter/DeepTFactor/deeptfactor/tf_running.py -i /home/justin/Master/Attribute_prediction/proteinrecord.fasta -o /home/justin/Master/Attribute_prediction/Promoter/DeepTFactor/deeptfactor/result -g cpu', returncode=0)

In [11]:
max_len = -1
seq_record = next(SeqIO.parse(open(gb_path), "gb")) #not sure if next() is neccesary here yet
locus_list = []
for feature in seq_record.features:
    if feature.type == "CDS":
        translation_key = 'translation'
        locus_key = 'old_locus_tag'

        if locus_key in feature.qualifiers:
            locus = ''.join(feature.qualifiers[locus_key])
        else:
            locus = ''.join(feature.qualifiers['locus_tag'])

        translation_key = 'translation'

        if translation_key not in feature.qualifiers:
            locus_list.append(locus)
            if 'pseudo' not in feature.qualifiers:
                print (locus)

        

        # if translation_key in feature.qualifiers:
        #     translated_seq = ''.join(feature.qualifiers['translation'])
        #     print (locus)
        #     if len(translated_seq) > max_len:
        #         max_len = len(translated_seq)
        #         max_locus = locus

# print(max_locus, max_len)


In [2]:
from Promoter.DeepTFactor.test_deeptfactor.deeptftest import deeptfactor

In [17]:
s = 'MTAFGNFVALCTHRAAANGDALAYHYLSPHGEDRALSFGALDRAARRVAARLAASGAPGDRVLIVCPQSLDYVSAFFGCLYGGFIAVPAYAPRNNHHFARLSKIIEDARPRVVMLCRKQYAAVHAFIERNPPLREVELVVVDELDDVEPGGCRPHAAARDDVAFLQYTSGSTGQAKGIMVSHGNLLANEEMIRTTCGNTPDSRAVFWLPLFHDMGLMTLLQGVYVGYPTYLMAPMDFLANPLRWLQAVSRVRATLTVAPNFAWQLCVEKIPPEQLDGLDLSSVTAAVNGSEPISVRTLDGFVARFGACGFRREAFRPSYGLAEATLLVTGSSGHAPDGVIEETMVSSAGRRTVFQRVGCGRPAAGCEVAIVERDTRAVRRDGELGEIWVKGPHVAQGYWNNPEQTAQTFGNRTADGAGPYLATGDLGFLHDGALVVTGRCKDVIILRGDNYYPSDLEASATAAHPALVPDGAAAFTLAGDEAGPDAAAAQALAVVAEVRRNTPPAQFAQIAAAIVERISAGYGLALERLVLIKERSIPKTSSGKVQRGAVRAELDAGQLKTLHDVRPGETGARAAAQPFAELHAMLAAARGDGLARIVDGFVLSQFAELMKTPFYALDLTRSVAALGLDSLALVNVQHRLGAALKIDVPAQLLFGDETLGELAAQIGALARRAWTDAPCAAASTDAAESADSADSAAAPQVPAVSDEAARAARNSPEAPAPALPAQADTDTGSDSTSPSTSTSTSPSAAAGWDATPGQAALWLIQQAARESHDYNEGFVAAVDGAFDPELFMLALARLAARRDALRVRFAEVDGRPVARDAHALEQSTLRCADDAAALQAARDALAAPFDLSRATWRAVLAKGPNATYVALALHHAIVDMWSFDILIAELKQLYDALARGADASLGAARGYREFAAWQHGWLNSPQATGHAAYWREQLRGMPARSAFPSTPIATPASGARAHRFAIDARTMGAVRALARRHGLTAYHVLFAAGALLLGRYAGQRDVVVGMPAHGRPRAADADTIGYFANVLPIRVALDDRRALADWLHDVKGAIVGGLLHQALPLPVLTEALAAERGAPDGALVQTVISLLAPSAARAVEAGADAGDRLSPFVLGEAGGRLALGDATLTSCESRHDRAPFDVAFTLIDDGGELAASIGYRADRHSAAMIERLAANYRCVLEALAHGACARLGDVPHVSAAEHALLRRSLTHRARAHAPAAGADGDSVVDRFRRTAARQPDALALAAAGRALTYGELARATDALGHMLVEHGVTPGDRVALAVGERAMQTRVALAILKVGAAYVPVDLANPPERLAYLLDDCGAKLVLTTRDDRPRLPATGANVVCADALDDAAAARHAGRPLPRVAIAAGQPAYCIYTSGSTGQPKGVLVTHGGLANLVDWHVDAFALDAGARASMLAGPGFDAAVWEIWPALCAGASLAEPAPDARHDVAELARWLDAHAISHCFMPTPLAEAFIAAAARPRALRFLLTGGDQLKARGRAGDGFQLINAYGPTENTVVTTSGAVEPASESAQGPAGPARLAPLPDIGAPIRGQALHILDAQLRPTPLGVSGELYVSGAGLALGYLNRPALTAERFVPAPDADAPDVPTPGARLYATGDLVRLDENGRLHFVGRADDQVQIRGFRVEPGEVEAVLATHPGVAQCKVIAFEREPSGKLLAAYVAGDASLTEAALRAFVDSRLPSYMRPAGFVIADALPLDANGKISRRALPPPTFAAADEAAFADPLEQAVANVYQEALGCGPVRADDDFFALGGHSLLVARVAAGIRARLGITVSIADLFAHSKVAALAAHLRGAAAAPIPPAVRLTRAGRPARVPLSPAQQRMWLLQALEPDSTDYHVCGALRVHGALDLERLAAALTQTVARHEALRTCFPRQDGQACQVVLPPAAVAVERLSVDIDADRASPAAHDAAVQARIDAWEAQRFDLETGPLLKAAYLPLAADDGYLLLNLHHIIADGRSIAILLDDIAARYADLREAGAGADAPGAPEAPHYADYCVWSRSEAAREREAASLPFWRTQLAGAWPAGLAALRPARTAAAQHAPGAARGDEIGDTLPADLARNVRAVAARWGVTESTVYLAAYGLLVAKLGGDADVLIGIAYAGRDLPDTADMIGMFVNVLPIRVALGDADAFEQQVRGWRDACVAAFRHAHVSYERMVELAGGDRRDGPGELVKTMFDFEEPGPPPRALGDARLSVERRADRSAKFDLTLRCRPDALGRMRIALNFRRAALSVERARGLLDAYRCVLEQVLQAPELTPARTRLVDAAQADALLRLGGGPAAPWAGAPVHRQIERIAHADPQALALAGEDGARIGYAVLNATANRLARRLRTLGVAPRDAVALCMRPGPSFALAALAVLKLGAAYVPIDPRYPDPRKLRIVADSGARLVVAEPDAAPAQAPDAVAPVWWDALAAQAATLPDSDLDIETAPDDLAYIVYTSGTTGAPKGVEIPHRGLANLCAWHARAYGLHDAPQSIRASQTAGIGFDAAVWEIWPYLCTGASVWFAPDAARQSSRHLEDWLSTRRITHCFAATPLAHAVLADGWLGSPSLAYLLTGGERLTRRPPAGARYRLFNHYGPSENSVVATAGEVAQGAGEPPSIGAALDNVRVYVLDRHGQLAPRGVPGELYIGGASLMRGYRSNDALTHARLVADPFAGVPDARMYRTGDLVCWNDAGELDYVGRADDQVKIRGHRIEPSEILHAVKSHAGVYDAAVTTLDHPQAGPQLVAYVVFDPAHGAGAPDRAARLKRAIAAQLPAFMVPTHVVELDALPLTSNGKIDYAALPAPRAANADARRTPLRTETERALAAIWSALLGEPVDDAHANFFALGGHSLLAARAVALIETRMAREIALKDFFAADDLAALARALDAASAHAGIPVAPAGAAIPLSPSQQRLWLVQAFNRGSVDYNVVGALRVSGALDAARLRAAVAAAVERHEILRTRIVDTEDGPRQRAPGADAREADLLQMNLTGNPPSMQDDFVRDYLARLAVEPFDLATGPLFKLVLIQTAADSGVLAASFHHIVVDAWSANVFLRDLLECYAAQCAQRAPRLAPLPVQYRDYSVWAQRELARSEDALRAFWRDYLHALPTHAPLPVAHHRAGAAPRDGRRLSIAWPADTLARLKALARREQASLYEVLIAAYFAWLHRLSGQSDLVVGMPYNDRGRAELENLVGFFVNVLPVRARVRPHLSYAALLRQVRDDLRRVYRHHALPFDRIAEQCANASGSLFQTMFDLDAQPLAHDARTADGDGLRAELLDGAPQAPIADLSVTLRETAQGLTLSCVYAAERFDAPAVERWLENFRALLDSVAADADQAVGRLAFLTANEQAIVERLRNPASVDHRGVLAQAGGANGPALLHRLVDHWATAAPANPALVSPARTLSFAALARSTDALAAVLREEGVRPGMVVGVEAAHSVDAVLGIIATIKAGAICLPVDTRLPPERLDAMIADSGCRHVLASAGAPLGRFDGKRLALDGAARHAGDAPAPAAEATPDHGVFLTYTSGTTGAPKASVLHHRGIVNYIGTVIERFGYTCGDRAMLFAPLTFDASLEEIFAPLCAGASLYIGDENVKRSVPALVDACRAQRISVLTLPTAYWRVLGEHLAANGGAAGLGAVRLVSIGGEKVTLEAIRQWHRATAGRIALYNIYGPSECSIGSIVDRIDVARALEDGEVYLHRPVANAHLHVLDACLNPVPADMPGELYIGGVGVAHGYHGRPALTAQRFVADPFAAAPGARLYRSGDRVRYDLEGRLHYLGRTDFQVKVDGIRVEPEEIQAVLEAHPDVAQAVVLAGEARHARNPLIGYVILEKAPARARAATGTDGAAFVDYLRARLPAHMVPAQVVVMDAFPLTTNQKVDRRALLLAANDAVVAAPALSPTETALLALWRELLGNDAFGVDDNFFSLGGSSLLAIRINSRVEAMYAVRLPVAALFDCPTVRALAVLVERLRASRDAAAPDQAGPPIERIDARATPLSGAQYRLWYLHQRDPDSTAYHLPDLLKLAGPLDAAALRTALAHTLDEHESLRTTFVVEGETPLQVIAARAQPRLDGHDLSALAPDARDAALDALVRRALDTPFDLARGPLATFALVTLAPDEHLLLSVFHHIVVDGWSVGLFQRSLARHYNAARGGRAAEPAADASARALQYRDFAAWHRRVLDGGERERQLDYWRAQLDGTLPVLQLPTDFKRPPQASGGGEAFSFAFDDALAARLQAVAQRHHVSMFMLLLAAYAALLARYARQDDLLIGVPALGRTRPEFESVVGFFVNTLVIRVRANPSTTFASLLEHVRATCLDAFDHQDVALEEIAADVNARRERDGTGLFQTMFSYQEADALEPAAFDGLVATSVEPEHRSAKFDLYLATWTHDGRLHGGFEFSTDLFEAATVRRMADHYRNLLAGVAARPDGALHALPLLGDDERAYQVRALNQASRPLPDGVYVRDLFARQAALHPARVAASCGEAALTYGELDRASDRVARNLLAAGARGEDLVGLLIGRNLDYLIAMLGVLKAGVAFTPMNPDDPAHKLDRIAELGDVRYVVHDAASAGRAQALTARATRLALDALSREPAAAPEFLPLTPASLAYVIYTSGSTGLPKGAMIEQRGMLNHLLAKIDDLAIGEDDVVAEMAVTTFDVSIWQYLVALLVGGRTAVMPGDAAWDPQQLFAQLDADGVTVFESVPSHMKILIDELEARPGRHRLGRVRVYVSNAEALTPALCARWFACAPHVPVVNTYGATECSDDTSHLWIREPLSGALPYVPIQGTLPNLTTYLLDERLEPVPIGVTGEVHIGGVGVGRGYLGDPARTARAFVPDPFSPRPGRRLYKTGDLARYRPDGTLEFLGREDFQVKIRGQRVEIGEVERAIGDHDNVRQAVVVAARDGKDRLYLLGYVIPHRHPAPTVPELRAFVAGRVASYMVPASFVLMDQFPLNANGKVDRKRLPKPADQDIFRRNEHVAPATQTEARLAELWRELLDVDEIGALDSFFELGGHSLLAAELTLRMRTLFGVALPLRTLFQSPQLRDVAAALDTLRTSADAAGPARVERLPARAHYELAPCQVPEWYAYQMDPSSPVYNISVADLFFTGKLNRDAFIAAWRAILERHDVLRVKFDYRDGAPIQIVDPAIDIRAENVFLDRTTLAGADAIDEANRLGARFGTAPFDFANGPLFRLHVASYAGDFHQLIFVVHHIIWDETSLINLMLELSELYNAHAAGRAANVPALEVGYFDYVQWTHQQLRCGAFDEHKRYWLDMYRTLPPPLDLPTDRARPNLMSYRGDALRTWLPRGIVRKIEAYLKQHDVTLFMLQLAILDCYLSRISGQRDFVIGCPIAGRADERLKPLLGLFATPMPIRCTIGENMTFGELLAQVSQRTLEAFEHYHYPSNQVIEQLQHEKDLSRPKLFSIMYGVQNNKTDLMGRLKFDGLTLSLENVVDTENKSSRFDLNFVVDQFGSDIMFSCIYNVDLFDASTIEQMLENMTALMDQVLDDPDQPLSRYTMVGANGNPPAALHGPRVDYDEAATMHALMAEQAARTPAATALVVEGVAYDYETLNRQANRLAHYLLSLGVTSGENVAVMLRPSFEMVVALYAILKVGGAFVPIGPQYPQKRVDAILRHAHARWALTHSGLRRAFAAFPYDVVCVDDVMGALGGYSDRNPPSVDPRQLAYILHTSGSTGTPKGIEIEHRGVVSMLADLQRTYCLDAHDRVLFHTPFTFDVFIQDVFWPLAYGARVVVMGDDALKSAHGLADVIERERVTLAQFVPAMLETLVDARERGEIAGLASLRQVICGAAALYRGLAERFARAFGCRLANHYGPTEVTVDASRFDCAEPYAGDTVPIGRPVGNASLHVLDAHLQPVPRGVIGEICVASPGLARRYLNDDEGTARAFVEIVVDGAPLRLYRTGDLGHCDRTGVVYFHGRADKQLKIRGNRVELDEIGSVLRSHPAIAAAALAYREDAAHGGRLVAYIEQTARDHQVPAEPGDAPLYGFTLEQRPELAAAAFALPPGADEAARELARRFPACQLVVTDRASTVTAFCQAVPLRIDATAQAGAPALERDAALRLAFAQQDDGVEPNALYVLDDAATGAPATSPAARAALLAPWRRLAAAFGLARLLRASAGGKPAEPADAGGQGAPAHAYLTRDAVRAWLRQSLPDYMIPDQVHFLPAIALTDSGKVDARNLPVIEVDEQPGRQAASTALQRELADLWERLLQVGQIGVTDDFFVLGGQSLKAIEMVAEVGRRYGVKIQLRQFYENPTIRYLETLLTERA'
list_1 = [s[i:i+1000] for i in range(len(s)-999)]
len (list_1)



5275

In [6]:
foo = deeptfactor("HKTLQITEQHFRRLYAGVPFDYFVYDSAKLGEVRSFASKSIVQIMIVTVAAINKKDVNTLYKDSEHTGGEKPIDLIRATHPIVIVDEPQSVDGGLEGRGKEALEAMRPLCTLRYSATHADKYQMLYRLDAIDAYERKLVKQIEIASATVEDAHNKPFVRVISIGSRRGAIAARVELDVATAAGDVERQTVSVSDGDDLERVARRAVYANFSIGEIHAARGAEYLVLRYPGGDAFLSVGDAYGDVDTHAIQREMIRRTIREHLDKELRLTPLGVKVLSLFFVDAVDKYRKYDRHGQPFKGDYALLFEDEYRRAAKLPEYRALFDGVDAGLAAEAVHDGYFSIDKKGGWTDTSDRSAGSRENAERAYGLIMKDKERLLSFDTPLKFIFSHSALKEGWDNPNVFQICTLRDIHSERERRQTIGRGLRLAVNQRGERVRGFDVNTLTVIAGESYEQFAENLQKEIEADTGIRFGIVETHQFAALPVPADDGSVQPLGVERSAALWMHLRDAGYLDARGRVQDTLRAALKLRALPLPDEFDALRALIVDMLRKLAGRLDVRNADERRHIALRRDAHGKAVYLGDAFRALWDRIRHRTTYRVNFDNARLIERCVAALKAAPAVTRARLQWRKADIAIDASGVEATETEDAGAIAIDEGEVELPDLLTELQDRTQLTRRTIAKVLIESGRLDEFPLNPQRFIALVAAALERCKRDALVDGIEYRLLGEAQVHALSLFESEPLTGYLSSMRRGAAKSIHEDVPCDTPAERAFVESLEQDDAVRLYAKLPGWFKIPTPLGSYSPDWAVLIAEGDGPRLYFVVESKSGVADGDLRADERRRVQCGAAHFRALEAVADNPARYVRARRADDLPTAAANARDAA", 'test')

print(foo)

False


/home/justin/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448216815/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [19]:
for seq in list_1:
    foo = deeptfactor(seq, 'test')

    if foo == 'True':
        break

print(foo)

False
